In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
#from finrl.apps import config

from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
#from finrl.drl_agents.stablebaselines3.models import DRLAgent # Deep Reinforcement Learning Agent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint
import sys
sys.path.append("../FinRL-Library")

import sys


/Users/henriquerio/miniconda3/lib/python3.8/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
ef = pd.ExcelFile("FIXEDDATA.xls") #used to get the sheets in our excel file
sheetnames = ef.sheet_names


In [3]:

df_combined = pd.DataFrame()
for sheet_name in ef.sheet_names:
    df = ef.parse(sheet_name)
    df['tic'] = sheet_name       # this adds `sheet_name` into the column `Week`
    df_combined = df_combined.append(df)

/var/folders/_k/2prywzdn6ng6b5g46h3r_mtc0000gn/T/ipykernel_15833/2518331846.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_combined = df_combined.append(df)
/var/folders/_k/2prywzdn6ng6b5g46h3r_mtc0000gn/T/ipykernel_15833/2518331846.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_combined = df_combined.append(df)
/var/folders/_k/2prywzdn6ng6b5g46h3r_mtc0000gn/T/ipykernel_15833/2518331846.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_combined = df_combined.append(df)
/var/folders/_k/2prywzdn6ng6b5g46h3r_mtc0000gn/T/ipykernel_15833/2518331846.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_combined = df_comb

In [4]:
df1 = df_combined
df1.head()

,Date,Open,High,Low,Close,Volume,tic
0,2020-01-01,4.497428,4.558976,4.478464,4.502792,578070570,ETC
1,2020-01-02,4.502639,4.508207,4.203510,4.255888,692082812,ETC
2,2020-01-03,4.249905,4.633809,4.203081,4.576237,853611122,ETC
3,2020-01-04,4.571729,4.732288,4.542132,4.726842,771746505,ETC
4,2020-01-05,4.773919,5.014711,4.771709,4.907899,929077856,ETC


In [5]:
from datetime import datetime
#df1['Date'] = str(df1['Date'])
#df1['Date']  =df1['Date'].apply(lambda x: datetime.strptime(x, "%d.%m.%Y") )
df1= df1.sort_values(by=['Date','tic'], ignore_index=True)

In [6]:
df1

,Date,Open,High,Low,Close,Volume,tic
0,2020-01-01,0.032832,0.033813,0.032704,0.033458,22948374,ADA
1,2020-01-01,4.216717,4.467644,4.204366,4.380158,107994513,ATOM
2,2020-01-01,204.671295,208.077515,203.205154,204.397537,1456113692,BCH
3,2020-01-01,13.730962,13.873946,13.654942,13.689083,172980718,BNB
4,2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,18565664997,BTC
...,...,...,...,...,...,...,...
12811,2021-12-12,19.945532,20.823475,19.278799,20.456585,580366114,LINK
12812,2021-12-12,157.925339,162.485016,153.610352,159.203964,870576845,LTC
12813,2021-12-12,0.091574,0.092408,0.089894,0.091136,845372145,TRX
12814,2021-12-12,48759.597656,49960.847656,48375.175781,49638.460938,199435671,WBTC


In [21]:
ratio_list = ['Open','High','Low','Close','Volume']

training = df1.iloc[1:12616]
trading = df1.iloc[12616:12816]

In [22]:
stock_dimension = len(training.tic.unique())
state_space = 1 + 2*stock_dimension + 5*stock_dimension

In [19]:
state_space

127

In [26]:
# Parameters for the environment
env_kwargs = {
    "hmax": 100, # -100<=K<=100 IT MEANS 100*2+1=201 ACTION SPACE
    "initial_amount": 100000,
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": ratio_list,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
    "num_stock_shares": 18

}
#Establish the training environment using StockTradingEnv() class
e_train_gym = StockTradingEnv(df = training, **env_kwargs)
env_train, _ = e_train_gym.get_sb_env()

KeyError: 0

5